In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import glob
import nd2reader
import numpy as np
from cytoolz import compose
import skimage
from skimage.color import rgb2gray
from skimage.feature import blob_dog, blob_log, blob_doh
from paulssonlab.projects.molecule_counting.segmentation import segment, invert
from paulssonlab.projects.molecule_counting.matriarch_stub import permute_labels
import itertools

# import cv2
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox
import signal
import time

mpl.rcParams["figure.figsize"] = (10, 10)

In [ ]:
segment_phase = compose(segment, invert)

In [ ]:
# Set up timers so that functions timeout and move on if something is weird and they don't run properly.
# Should diagnose this issue properly eventually, but for testing this should be ok.
class TimeoutException(Exception):  # Custom exception class
    pass


def timeout_handler(signum, frame):  # Custom signal handler
    raise TimeoutException


# Change the behavior of SIGALRM
signal.signal(signal.SIGALRM, timeout_handler)

In [ ]:
filenames = glob.glob("/n/scratch2/jqs1/200228/*.nd2") + glob.glob(
    "/n/scratch2/jqs1/200305/*.nd2"
)
for i in range(len(filenames)):
    print(i, filenames[i])

In [ ]:
file = filenames[2]
print("file", file)
nd2 = nd2reader.ND2Reader(file)
fov_index = 20
phase_img = nd2.get_frame_2D(v=0, c=0, z=fov_index)
fluor_img = nd2.get_frame_2D(v=0, c=1, z=fov_index)
# phase_img = nd2.get_frame_2D(v=0, c=0, z=0)[1100:3000,500:2250]
# fluor_img = nd2.get_frame_2D(v=0, c=1, z=0)[1100:3000,500:2250]
fig, axes = plt.subplots(1, 2, sharex=True, sharey=True)
ax = axes.ravel()
ax[0].imshow(phase_img)
ax[1].imshow(fluor_img, cmap="RdGy")

In [ ]:
seg = segment_phase(phase_img)
fig, ax = plt.subplots()
# ax.imshow(seg)
ax.imshow(permute_labels(seg))

In [ ]:
def get_indices_in_r(ref_indices, r):
    # ref_indices is a list of the indices that define the location of the reference point.
    # r is the radius within which we want to collect indices.
    ranges = np.arange(np.floor(-r), np.ceil(r) + 1)
    rel_points = np.where(
        (ranges[np.newaxis, :]) ** 2 + (ranges[:, np.newaxis]) ** 2 < r**2
    )
    points = [
        rel_points[0] + ref_indices[0] - np.ceil(r),
        rel_points[1] + ref_indices[1] - np.ceil(r),
    ]
    return points

In [ ]:
def rdf_single_ref_point_single_cell(cell, ref_point, time_allowed=80, start_r=1, dr=1):
    #     returns rdf for a single reference point in a single cell.
    #     cell is a masked array, where the mask is False where the cell is, and True everywhere else.
    #     cell.data is the fluorescence intensity array.
    #     ref_point is a tuple of 2 arrays, corresponding to the indices of the location of the 'nucleus'
    # rho is the average 'particle' density, where a particle corresponds to an intensity unit.
    rho = np.mean(cell)
    r = start_r
    r_values = []
    #         masked_norm_fluor_img.mask is False where the cell is, and True outside.
    #         mask = np.copy(cell.mask)
    gr_values = []
    previous_r_vol = 0
    while np.count_nonzero(cell.mask == 0) > 0:
        #         print('points remaining',np.count_nonzero(cell.mask==0))
        #             While the cell mask still has 'False' values i.e. not all values have been used in the
        #             pair correlation function calculation somewhere.
        #             Get indices of the points within the current radius (r) of the ref point.
        indices_in_r = get_indices_in_r(ref_point, r)
        flat_indices_in_r = [item for sublist in indices_in_r for item in sublist]
        img_shape = np.shape(cell)
        #         Remove indices pairs which fall outside the image.
        if any(flat_indices_in_r >= np.min(img_shape)):
            indices_to_delete_0 = np.where(indices_in_r[0] >= img_shape[0])[0]
            indices_to_delete_1 = np.where(indices_in_r[1] >= img_shape[1])[0]
            indices_to_delete = np.concatenate(
                (indices_to_delete_0, indices_to_delete_1)
            )
            indices_to_delete = np.unique(indices_to_delete)
            indices_in_r[0] = np.delete(indices_in_r[0], indices_to_delete)
            indices_in_r[1] = np.delete(indices_in_r[1], indices_to_delete)
        # Calculate numerator (gr_num) and volume element (gr_vol)
        # gr_num should be the sum total of intensities of points in the current shell.
        #             Since the cell is masked, we don't need to worry about the non-cell regions.
        gr_num = np.sum(
            cell[list(map(int, indices_in_r[0])), list(map(int, indices_in_r[1]))]
        )
        #         print('grnum',gr_num)
        # gr_vol should be the number of points in the current shell.
        #             Note that the mask masks both the background and the previous regions that we have considered,
        #             so we simply subtract these from our list of indices in the current radius.
        gr_vol = len(indices_in_r[0]) - np.count_nonzero(
            cell.mask[list(map(int, indices_in_r[0])), list(map(int, indices_in_r[1]))]
        )
        #         print('gr vol',gr_vol)
        #             update the 'previous_r_vol' for the next radius shell.
        previous_r_vol = gr_vol
        #             Update the cell mask so that the values we have now included are not included
        #             in the next dr calculation.
        cell.mask[
            list(map(int, indices_in_r[0])), list(map(int, indices_in_r[1]))
        ] = True
        #             print('number of non masked values',np.count_nonzero(cell.mask==0))
        gr_values.append(gr_num / gr_vol)
        r_values.append(r)
        r += dr
    # Since rho is constant for all r, divide all at the same time to save on number of computations.
    gr_values = np.array(gr_values)
    gr_values = np.divide(gr_values, rho)
    return r_values, gr_values

In [ ]:
def rdf_multi_ref_point_single_cell(
    cell, ref_points, time_allowed=80, start_r=1, dr=1, intensity_weighted=True
):
    #     cell is a masked array, where the mask is False where the cell is, and True everywhere else.
    #     returns r_values, gr_values for one cell, where gr_values is the average gr over the
    #     ref_points.
    #     ref_points should be a tuple of 2 arrays, where the first array is the axis 0
    #     indices, and second array is the axis 1 indices, as we would get from np.where
    max_r_values = []
    all_gr_values = []
    if intensity_weighted:
        total_ref_point_intensity = np.sum(
            cell[list(map(int, ref_points[0])), list(map(int, ref_points[1]))]
        )
        num_ref_points = len(ref_points[0])
    #         total_cell_intensity = np.sum(cell)
    #         pixels_in_cell = np.count_nonzero(cell.mask == 0)
    for i in range(len(ref_points[0])):
        cell_for_analysis = np.ma.copy(cell)
        ref_point_0, ref_point_1 = ref_points[0][i], ref_points[1][i]
        r_values, gr_values = rdf_single_ref_point_single_cell(
            cell_for_analysis,
            (np.array(ref_point_0), np.array(ref_point_1)),
            time_allowed=80,
            start_r=start_r,
            dr=dr,
        )
        if intensity_weighted:
            gr_values *= cell.data[ref_point_0][ref_point_1]
        if len(r_values) > len(max_r_values):
            max_r_values = r_values
        all_gr_values.append(gr_values)
    gr_values_length_corrected = list(
        itertools.zip_longest(*all_gr_values, fillvalue=np.nan)
    )
    mean_gr_values = [np.nanmean(i) for i in gr_values_length_corrected]
    if intensity_weighted:
        #         mean_gr_values = np.multiply(pixels_in_cell,mean_gr_values/total_cell_intensity)
        mean_gr_values = np.multiply(
            num_ref_points, mean_gr_values / total_ref_point_intensity
        )
    return max_r_values, mean_gr_values

In [ ]:
def rdf(seg, fluor_img, ref_point_val = 'max', cell_indices=[0], time_allowed=80, 
        savetxt='rdf',start_r=1,dr=1, intensity_weighted = True):
# Seg is the segmentation array.
#     Get the background fluorescence of the image, to subtract from all cell fluorescence.
    min_fluor = np.amin(fluor_img)
    # Get number of cells in the image.
    cells = np.amax(seg)
    if cell_indices == 'all':
        cell_indices = np.arange(cells)
# all_r_values will be a list of lists of r values, while all_gr_values will be a list of lists, with
# the corresponding g(r) (pair correlation function) values. The overall function will return
# all_r_values, all_gr_values.
    all_r_values = []
    all_gr_values = []
    for cell_index in cell_indices:
        print('cell index',cell_index)
        if cell_index>cells:
            print('Error in rdf: cell_index larger than number of cells in fov.')
            pass
#         Mask the fluorescence image everywhere except for the cell.
        cell = np.ma.masked_where(seg != cell_index + 1, fluor_img)
        # Normalise the intensities of the cell.
#         mean_int_cell = np.mean(cell)
        cell -= min_fluor
#         ref_point is the 'nucleus' i.e. the point relative to which the pair
#         correlation function is to be calculated. For now, we set it to the 
#         maximum fluorescence intensity point of the cell.
        if ref_point_val == 'max':
            ref_point = np.where(cell == np.amax(cell))
#         If there is more than one maximum fluorescence point, take the first one of these.
            if len(ref_point[0])>1:
                ref_point = (np.array([ref_point[0][0]]),np.array([ref_point[1][0]]))
        elif ref_point_val == 'all':
            ref_point = np.where(cell.mask == 0)
        elif ref_point_val[0] == 'sample':
            points_in_cell = np.where(cell.mask == 0)
            ref_point = (points_in_cell[0][::ref_point_val[1]],points_in_cell[1][::ref_point_val[1]])
        elif ref_point_val[0] == 'max':
            
        else:
            ref_point = ref_point_val
        r_values, gr_values = rdf_multi_ref_point_single_cell(cell, ref_point, 
                                                              time_allowed=80, start_r=start_r, dr=dr,
                                                             intensity_weighted = intensity_weighted)
        all_gr_values.append(gr_values)
        all_r_values.append(r_values)
    if savetxt:
        all_r_to_save = np.concatenate(all_r_values).ravel()
        all_gr_to_save = np.concatenate(all_gr_values).ravel()
        np.savetxt(savetxt, np.transpose(np.array([all_r_to_save,all_gr_to_save])))
    return all_r_values,all_gr_values

In [ ]:
# Test using seg as the fluor image, since this should return g(r) = 1 for all r.
# Everything is 1 in this case
# r_values_list, gr_values_list = rdf(seg,seg,cell_indices=np.arange(6),
#                                     ref_point_val=['sample',10],dr=10,
#                                    intensity_weighted=True)
# Everything is also 1 in this case
# r_values_list, gr_values_list = rdf(seg,seg,cell_indices=np.arange(6),
#                                     ref_point_val=['sample',10],dr=10,
#                                    intensity_weighted=False)

print("20200626/sfGFP_100x_fov20/refmax_dr1_intweighttrue")
rdf(
    seg,
    fluor_img,
    cell_indices="all",
    ref_point_val="max",
    dr=1,
    intensity_weighted=True,
    savetxt="20200626/sfGFP_100x_fov20/refmax_dr1_intweighttrue",
)

# print(r_values_list)
# print(gr_values_list)

In [ ]:
def get_cell_rdfs_from_txt(filename, start_r):
    # start_r is the start value of r in that dataset, so we can identify where each cell dataset starts.
    values = np.genfromtxt(filename)
    all_r_values = values[:, 0]
    all_gr_values = values[:, 1]
    start_indices = np.where(all_r_values == start_r)[0]
    r_values_list = []
    gr_values_list = []
    for i, val in enumerate(start_indices):
        if i != len(start_indices) - 1:
            r_values_list.append(all_r_values[val : start_indices[i + 1]])
            gr_values_list.append(all_gr_values[val : start_indices[i + 1]])
        else:
            r_values_list.append(all_r_values[val:])
            gr_values_list.append(all_gr_values[val:])
    return r_values_list, gr_values_list

In [ ]:
# Function to plot gr from file
def plot_gr_from_file(
    filename,
    start_r,
    ax=None,
    savefig=None,
    indices_to_plot=None,
    legend=False,
    title=None,
):
    r_values_list, gr_values_list = get_cell_rdfs_from_txt(filename, start_r)
    if not ax:
        fig, ax = plt.subplots()
    if not indices_to_plot:
        indices_to_plot = np.arange(len(r_values_list))
    for i in indices_to_plot:
        ax.plot(r_values_list[i], gr_values_list[i])
    if legend:
        ax.legend([i for i in indices_to_plot])
    if title:
        ax.set_title(title)
    ax.axhline(y=1, ls=":", color="k")
    if savefig:
        fig.savefig(savefig)

In [ ]:
# Plot grs from file
# fig,axes = plt.subplots(2,2,figsize=(20,20))
# ax = axes.ravel()
plot_gr_from_file(
    "20200626/sfGFP_100x_fov20/refmax_dr1_intweighttrue",
    1,
    savefig="20200626/sfGFP_100x_fov20/refmax_dr1_intweighttrue",
)
plot_gr_from_file(
    "20200626/sfGFP_100x_fov20/refmax_dr1_intweighttrue",
    1,
    savefig="20200626/sfGFP_100x_fov20/refmax_dr1_intweighttrue_foci",
    indices_to_plot=[1, 2, 11, 25, 27, 28, 33, 52],
    legend=True,
    title="Foci",
)
plot_gr_from_file(
    "20200626/sfGFP_100x_fov20/refmax_dr1_intweighttrue",
    1,
    savefig="20200626/sfGFP_100x_fov20/refmax_dr1_intweighttrue_maybefoci",
    indices_to_plot=[5, 8, 14, 20, 22, 30, 32, 40, 55],
    legend=True,
    title="Maybe foci",
)
plot_gr_from_file(
    "20200626/sfGFP_100x_fov20/refmax_dr1_intweighttrue",
    1,
    savefig="20200626/sfGFP_100x_fov20/refmax_dr1_intweighttrue_nofoci",
    indices_to_plot=[0, 3, 4, 6, 9, 13, 23, 29, 31, 36, 37, 44, 51, 58],
    legend=True,
    title="No foci",
)

In [ ]:
rough_sqrt = int(np.ceil(np.sqrt(62)))
fig, axes = plt.subplots(rough_sqrt, rough_sqrt, figsize=(25, 25))
ax = axes.ravel()
i = 0
for cell_index in range(62):
    cell_indices = np.where(seg == cell_index + 1)
    cell = np.ma.masked_where(seg != cell_index + 1, fluor_img)
    #     fig, ax = plt.subplots(figsize=(5,5))
    ax[i].imshow(
        cell[
            int(np.min(cell_indices[0])) : int(np.max(cell_indices[0])) + 1,
            int(np.min(cell_indices[1])) : int(np.max(cell_indices[1])) + 1,
        ]
    )
    ax[i].set_title(cell_index)
    i += 1
plt.tight_layout()
fig.savefig("20200626/sfGFP_100x_fov20/cells")

In [ ]:
# To check:
#     single point vs gradual gradient (i.e. size of the blob)
#     gradient of the intensity of the blob (i.e. one clear blob and no background or more diffuse blob)
#     different cell shape
#     position of blob within cell
#     intensity of the blob vs background of cell
# Test on 2d Gaussian and hat function